In [2]:
#!/usr/bin/env python
######## MÉMO pour moi
 # But: faire des recommandations de films à un utilisateur en se basant sur son historique et ses préférences.
    
 # On donne un utilisateur en entrée
#----
 # On regroupe ceux qu'il a noté par le Genre de film ave la moyenne de score de chaque Genre

#  | Genre | Score |
#|1|
#|2|
#|3|

 # Le prog va trié la liste des films qu'il n'a pas vu par le Genre ayant le meilleur Score
#----- 
# En sortie nous avons une liste de films qu'il n'a pas noté et étant de la catégorie qu'il aime le plus

#### IMPORT #############

import numpy as np
import pandas as pd
import sys

In [3]:
##### DEF DE FONCTIONS #######

def print_list(liste):
	for i in liste:
		print(i)


def print_list_of_list(liste):
	for i in liste:
		print_list(i)

def liste_TO_dataFrame(liste,columms):
	return liste_TO_dataFrame2(liste,columms)

def liste_TO_dataFrame2(liste,columms):
    assert len(columms) == len(liste[0])
    colList = [ [] for _ in columms]
    for i in liste:
        for j in range(0,len(columms)):
            colList[j].append(i[j])
    d = {}
    for i in range(0,len(colList)):
        d[columms[i]] = pd.Series(colList[i])
    df = pd.DataFrame(d)
    return df

def obtenirListe(fichierAOuvrir):
    try:
        print("[INFO]Ouverture du fichier",fichierAOuvrir,end='\n')
        f = open(fichierAOuvrir, 'r')
        # là ça fait une liste de liste
        l = [ list(line.strip('\n').split('::')) for line in f ]
    except Exception as e:
        print("le problème vient du fichier : ",fichierAOuvrir)
        raise e
    else:
        print("[INFO]Réussi",end='\n')
    return l

def recupIndexFilmReco(tabse):
	maxi=0
	index=0
	for i in range(0,len(tabse)):
		if tabse[i]>maxi:
			maxi=tabse[i]
			index=i
	return index


In [4]:
pathToRoot = "./"
#### DATA #####

	###### Pour les ratings 

lRatings = obtenirListe(pathToRoot+'Data/ratings.dat')
colRatings = ['UserID','MovieID','Rating','Timestamp']

dfRatings = liste_TO_dataFrame2(lRatings,colRatings)
#print(dfRatings)

	####### Pour les movies

lMovies = obtenirListe(pathToRoot+'Data/movies.dat')
colMovies = ['MovieID','Title','Genres']

dfMovies = liste_TO_dataFrame2(lMovies,colMovies)
#print(dfMovies)

	######## Pour les users

lMovies = obtenirListe(pathToRoot+'Data/users.dat')
colMovies = ['UserID','Gender','Age','Occupation','Zip-code']

dfUser = liste_TO_dataFrame2(lMovies,colMovies)
#print(dfUser)

[INFO]Ouverture du fichier ./Data/ratings.dat
[INFO]Réussi
[INFO]Ouverture du fichier ./Data/movies.dat
[INFO]Réussi
[INFO]Ouverture du fichier ./Data/users.dat
[INFO]Réussi


In [5]:
idUser = str(input("donne moi un utilisateur par son id : "))

donne moi un utilisateur par son id : 9


In [6]:
 # On donne un utilisateur en entrée
#----
 # On regroupe ceux qu'il a noté par le Genre de film avec la moyenne de score de chaque Genre

#  | Genre | Score |
#|1|
#|2|
#|3|

 # Le prog va trié la liste des films qu'il n'a pas vu par le Genre ayant le meilleur Score
#----- 
# En sortie nous avons une liste de films qu'il n'a pas noté et étant de la catégorie qu'il aime le plus

# Créer une nouvelle fonction qui détermine si la valeur en paramètre est manquante:
#def num_missing(x):
#    return sum(x.isnull())

# On applique cette fonction pour chaque colonne:
#print("Valeurs manquantes par colonne:")
#print(data.apply(num_missing, axis=0)) #axis=0 définit que la fonction sera bien appliquée sur chaque colonne

# Puis application pour chaque ligne:
#print("\nValeurs manquantes par ligne:")
#print(data.apply(num_missing, axis=1).head()) #axis=1 définit que la fonction sera bien appliquée sur chaque ligne


In [8]:
colGenreScore = ['Genre','Score']

ratingsOfUser = dfRatings.loc[dfRatings['UserID'] == idUser, colRatings[:3]]
print(type(ratingsOfUser['MovieID']))
#moviesOfUser = dfMovies.loc[dfMovies['Movies'] == ratingsOfUser['MoviesID'][i]]
print(type(ratingsOfUser))
print(ratingsOfUser)

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
    UserID MovieID Rating
693      9    2268      4
694      9    1466      4
695      9    1393      3
696      9     861      2
697      9    1682      4
..     ...     ...    ...
794      9    2762      4
795      9    1961      5
796      9    2692      4
797      9    1310      3
798      9     428      3

[106 rows x 3 columns]


In [42]:
# Pour chque ligne prendre le film, joindre avec dfMovies
# grouper par Genre de Movies
#print(dfMovies)
dfTEST = dfMovies
dfTEST2 = ratingsOfUser

# del dfTEST['Title']
print(dfTEST)

#del dfTEST2['UserID']
print(dfTEST2)

dfGenres = dfMovies.groupby(['MovieID'])
print(dfGenres)

     MovieID                        Genres
0          1   Animation|Children's|Comedy
1          2  Adventure|Children's|Fantasy
2          3                Comedy|Romance
3          4                  Comedy|Drama
4          5                        Comedy
...      ...                           ...
3878    3948                        Comedy
3879    3949                         Drama
3880    3950                         Drama
3881    3951                         Drama
3882    3952                Drama|Thriller

[3883 rows x 2 columns]
    MovieID Rating
693    2268      4
694    1466      4
695    1393      3
696     861      2
697    1682      4
..      ...    ...
794    2762      4
795    1961      5
796    2692      4
797    1310      3
798     428      3

[106 rows x 2 columns]


In [66]:
result = pd.concat([dfTEST, dfTEST2], axis=1, join='inner')
del result['MovieID']

In [67]:
print(result)

                   Genres Rating
693                 Drama      4
694                 Drama      4
695      Action|Adventure      3
696                Comedy      2
697                 Drama      4
..                    ...    ...
794        Comedy|Romance      4
795                 Drama      5
796                 Drama      4
797        Comedy|Romance      3
798  Adventure|Children's      3

[106 rows x 2 columns]


In [69]:
resultFinal = result.groupby(by='Genres')

In [70]:
print(resultFinal)